In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import matplotlib.pyplot as plt

In [2]:
# GPU Configuration
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# Data Augmentation for Training
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [4]:
training_set = train_datagen.flow_from_directory(
    "Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold5_AUG/Train/",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
)

Found 7532 images belonging to 6 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)
validation_set = test_datagen.flow_from_directory(
    "Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold5_AUG/Train/",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
)

Found 7532 images belonging to 6 classes.


In [6]:
# Custom F1 Score Metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Update the precision and recall for each batch
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        # Calculate F1 score as the harmonic mean of precision and recall
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [7]:
# Build the CNN-LSTM Model
cnn_lstm = tf.keras.models.Sequential()

In [8]:
# Flatten and Reshape for LSTM
cnn_lstm.add(tf.keras.layers.Flatten())
cnn_lstm.add(tf.keras.layers.Reshape((16, -1)))  # Reshape to (time_steps, features)

In [9]:
# LSTM Layer
cnn_lstm.add(tf.keras.layers.LSTM(units=64, activation='tanh', return_sequences=False))

In [10]:
# Fully Connected Layers
cnn_lstm.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn_lstm.add(tf.keras.layers.Dense(6, activation='softmax'))  # 6 classes

In [11]:
# Compile the model with Precision, Recall, and F1 Score
cnn_lstm.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Use categorical_crossentropy for one-hot encoded labels
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score(name='f1_score')  # Add custom F1 score metric
    ]
)

In [12]:
# Train the model
history = cnn_lstm.fit(
    x=training_set,
    validation_data=validation_set,
    epochs=100
)

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.3783 - f1_score: 0.0673 - loss: 1.6183 - precision: 0.4289 - recall: 0.0371

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


236/236 ━━━━━━━━━━━━━━━━━━━━ 72s 279ms/step - accuracy: 0.3784 - f1_score: 0.0674 - loss: 1.6182 - precision: 0.4294 - recall: 0.0371 - val_accuracy: 0.3537 - val_f1_score: 0.1179 - val_loss: 1.5746 - val_precision: 0.6653 - val_recall: 0.0647
Epoch 2/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 55s 232ms/step - accuracy: 0.3961 - f1_score: 0.1695 - loss: 1.5538 - precision: 0.5770 - recall: 0.0995 - val_accuracy: 0.4052 - val_f1_score: 0.2249 - val_loss: 1.5288 - val_precision: 0.5820 - val_recall: 0.1394
Epoch 3/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 55s 233ms/step - accuracy: 0.4008 - f1_score: 0.2038 - loss: 1.5348 - precision: 0.6012 - recall: 0.1228 - val_accuracy: 0.4153 - val_f1_score: 0.2719 - val_loss: 1.5050 - val_precision: 0.5867 - val_recall: 0.1770
Epoch 4/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 55s 232ms/step - accuracy: 0.4214 - f1_score: 0.2304 - loss: 1.5025 - precision: 0.5919 - recall: 0.1433 - val_accuracy: 0.4361 - val_f1_score: 0.2539 - val_loss: 1.4568 - val_precision: 0.6627 - val_rec

In [13]:
# Save Loss plot
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('loss_plot.png')  # Save the plot as a PNG file
plt.close()  # Close the figure to avoid memory issues

# Save Accuracy plot
plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('accuracy_plot.png')  # Save the plot as a PNG file
plt.close()  # Close the figure

# Save Precision plot
plt.figure()
plt.plot(history.history['precision'], label='Train Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.legend()
plt.title("Precision")
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.savefig('precision_plot.png')  # Save the plot as a PNG file
plt.close()  # Close the figure

# Save Recall plot
plt.figure()
plt.plot(history.history['recall'], label='Train Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')
plt.legend()
plt.title("Recall")
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.savefig('recall_plot.png')  # Save the plot as a PNG file
plt.close()  # Close the figure

# Save F1 Score plot
plt.figure()
plt.plot(history.history['f1_score'], label='Train F1 Score')
plt.plot(history.history['val_f1_score'], label='Validation F1 Score')
plt.legend()
plt.title("F1 Score")
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.savefig('f1_score_plot.png')  # Save the plot as a PNG file
plt.close()  # Close the figure

In [14]:
cnn_lstm.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 12288)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 16, 768)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │         213,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 667,028 (2.54 MB)

 Trainable params: 222,342 (868.52 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 444,686 (1.70 MB)